In [ ]:
import numpy as np
import random
import math
import pandas as pd
import copy

def read_neighborhoods():
    temp= pd.read_excel('./ProvinceNeighbours.xlsx')
    np_array=temp.to_numpy()
    return np_array

def TS_for_map_coloring(neighborhoods,num_colors):
    # parameters
    k=num_colors # number of colors
    tt=10 # tabu tenure
    threshold=150 # number of non-improving moves need to terminating search
    best_so_far_value=0
    best_so_far_solution=[]
    best_tabu_neighbor=[-1,-1,0] # (color,province,value)
    best_non_tabu_neighbor=[-1,-1,0]

    # initial solution
    current=[]
    for i in range(31):
        current.append(random.randint(0,k-1))
    current_value=0
    for i in range(31):
        for j in range(31):
            if i>=j and neighborhoods[i][j+1]==1:
                if current[i]==current[j]:
                    current_value+=1
    best_so_far_value=current_value
    best_so_far_solution=current
    best_non_tabu_neighbor[2]=best_tabu_neighbor[2]=current_value

    # TS
    # construct initial tabu list
    tabu_table=np.zeros((k,31))

    count=0
    while count<threshold and best_so_far_value != 0:
        for i in range(k):
            for j in range(31):
                next=copy.deepcopy(current)
                next[j]=i
                next_value=0
                for g in range(31):
                    for h in range(31):
                        if g>=h and neighborhoods[g][h+1]==1:
                            if next[g]==next[h]:
                                next_value+=1
                
                if(tabu_table[i][j]==0): # the neighbor is non tabu
                    if best_non_tabu_neighbor[0]==-1 and best_non_tabu_neighbor[1]==-1:
                        best_non_tabu_neighbor[2]=next_value
                        best_non_tabu_neighbor[0]=i
                        best_non_tabu_neighbor[1]=j

                    elif next_value<best_non_tabu_neighbor[2]:
                        best_non_tabu_neighbor[2]=next_value
                        best_non_tabu_neighbor[0]=i
                        best_non_tabu_neighbor[1]=j

                else: # the neighbor is tabu
                    if best_tabu_neighbor[0]==-1 and best_tabu_neighbor[1]==-1:
                        best_tabu_neighbor[2]=next_value
                        best_tabu_neighbor[0]=i
                        best_tabu_neighbor[1]=j

                    elif next_value<best_tabu_neighbor[2]:
                        best_tabu_neighbor[2]=next_value
                        best_tabu_neighbor[0]=i
                        best_tabu_neighbor[1]=j

        if best_tabu_neighbor[2]<best_non_tabu_neighbor[2] and best_tabu_neighbor[2]<best_so_far_value:
            best_so_far_value=best_tabu_neighbor[2]
            current[best_tabu_neighbor[1]]=best_tabu_neighbor[0]
            best_so_far_solution=current
            tabu_table[best_tabu_neighbor[0]][best_tabu_neighbor[1]]=tt
            count=0
            
        else:
            current[best_non_tabu_neighbor[1]]=best_non_tabu_neighbor[0]
            tabu_table[best_non_tabu_neighbor[0]][best_non_tabu_neighbor[1]]=tt
            if best_non_tabu_neighbor[2]<best_so_far_value:
                best_so_far_value=best_non_tabu_neighbor[2]
                best_so_far_solution=current
                count=0
            else:
                count+=1

    return best_so_far_solution , best_so_far_value

neighbors=read_neighborhoods()
use_3_colors=TS_for_map_coloring(neighbors,3)
use_4_colors=TS_for_map_coloring(neighbors,4)

print("If we use 3 colors the value of objective function is ", use_3_colors[1]," (it must be 0)")
for i in range(31):
    print(neighbors[i][0],": color ", use_3_colors[0][i])

print('\n')


print("If we use 4 colors the value of objective function is ", use_4_colors[1]," (it must be 0)")
for i in range(31):
    print(neighbors[i][0],": color ", use_4_colors[0][i])


# it sometimes solves the problem with 4 colors
